In [14]:
# Imports
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas
import time

In [15]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [12]:
url_fandom = "https://tbate.fandom.com/wiki/The_Beginning_After_The_End_Wiki"

In [21]:
# Load Fandom Page
driver.get(url_fandom)

try:
    driver.find_element(By.XPATH, '//div[text()="ACEITAR"]').click()
except Exception as e:
    pass
    try:
        driver.find_element(By.XPATH, '//div[text()="ACCEPT"]').click()
    except Exception as e:
        pass

In [23]:
time.sleep(3)
novel_btn = driver.find_element(By.XPATH, '//*[@id="NavBtns"]/li[5]')
novel_btn.click()
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="SliderWrapper"]/li[5]/a').click()
time.sleep(5)
driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/ul/li/i/a').click()

In [24]:
volumes_titles = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/h2')
volumes_chapters = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/ul')

In [30]:
list_chapter_per_volume = []
for volume_title, volume_chapter in zip(volumes_titles, volumes_chapters):
    list_chapter_per_volume.append({volume_title.text: volume_chapter.text.splitlines()})
columns = [[*volume.keys()][0] for volume in list_chapter_per_volume]
df_caps_per_volumes = pandas.DataFrame([volume[col]  for col, volume in zip(columns, list_chapter_per_volume)]).T
df_caps_per_volumes.columns = columns

In [44]:
last_titles = volumes_titles[len(volumes_chapters):]
last_volumes = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/table/tbody')

In [46]:
columns = []
caps_vol = []
for index, (title, chapters) in enumerate(zip(last_titles, last_volumes)):
    columns.append(title.text)
    chapter_name = chapters.find_elements(By.XPATH, f'//*[@id="mw-content-text"]/div/table[{index+1}]/tbody/tr/td[1]')
    caps_vol.append([name.text.replace("\n", ": ") for name in chapter_name])
df_caps2 = pandas.DataFrame(caps_vol).T
df_caps2.columns = columns[:2] + ["Exclusive Chapters"]

merged = df_caps_per_volumes.join(df_caps2)

In [49]:
columns = [column.split(":")[0] for column in merged.columns]
merged.columns = columns

In [51]:
merged.to_json("./Data/caps_per_volumes.json")